# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [217]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

# loading data from python.db database
con = sqlite3.connect('data/python.db')

# Read 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql("SELECT * FROM school", con)
student_data = pd.read_sql("SELECT * FROM student", con)


In [8]:
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [11]:
# Create a District Summary
school_data_complete.describe()

,id_x,Student ID,reading_score,math_score,id_y,School ID,size,budget
count,39170.000000,39170.000000,39170.00000,39170.000000,39170.000000,39170.000000,39170.000000,3.917000e+04
mean,19584.500000,19584.500000,81.87784,78.985371,6.978172,6.978172,3332.957110,2.117241e+06
std,11307.549359,11307.549359,10.23958,12.309968,4.444329,4.444329,1323.914069,8.749987e+05
min,0.000000,0.000000,63.00000,55.000000,0.000000,0.000000,427.000000,2.480870e+05
25%,9792.250000,9792.250000,73.00000,69.000000,3.000000,3.000000,1858.000000,1.081356e+06
50%,19584.500000,19584.500000,82.00000,79.000000,7.000000,7.000000,2949.000000,1.910635e+06
75%,29376.750000,29376.750000,91.00000,89.000000,11.000000,11.000000,4635.000000,3.022020e+06
max,39169.000000,39169.000000,99.00000,99.000000,14.000000,14.000000,4976.000000,3.124928e+06


In [13]:
# Total number of schools
total_schools = school_data_complete["school_name"].nunique()
print(f'Total number of schools: {total_schools}')

Total number of schools: 15


In [15]:
# Total number of students
total_students = len(school_data_complete["student_name"])
print(f'Total number of students: {total_students}')

Total number of students: 39170


In [16]:
# Total budget
total_budget = school_data_complete["budget"].sum()
print(f'Total school budget: {total_budget}')

Total school budget: 82932329558


In [19]:
# Average math score
avg_math = school_data_complete["math_score"].mean()
print(f'Average math score: {avg_math}')

Average math score: 78.98537145774827


In [21]:
# Average reading score
avg_read = school_data_complete["reading_score"].mean()
print(f'Average reading score: {avg_read}')

Average reading score: 81.87784018381414


In [23]:
# Overall average score
overall_score = (avg_math + avg_read)/2
print(f'Overall average score: {overall_score}')

Overall average score: 80.43160582078121


In [25]:
# Percentage of passing math (70 or greater)
pass_math = len(school_data_complete.loc[school_data_complete["math_score"] >= 70])
overall_math = len(school_data_complete)
pass_math_ratio = round(pass_math / overall_math * 100, 2)
print(f'Percentage passing math: {pass_math_ratio}%')

Percentage passing math: 74.98%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [31]:
#  Sort and display the top five schools in overall passing rate
school_summary = school_data_complete.groupby(['school_name', 'type']).agg(
    total_student=('Student ID', 'size'),
    total_school_budget=('budget', 'sum'),
    avg_math_score=('math_score', 'mean'),
    avg_read_score=('reading_score', 'mean'),
    pass_math=('math_score', lambda x: (x > 70).sum()),
    pass_read=('reading_score', lambda x: (x > 70).sum())
).reset_index()

school_summary['per_student_budget'] = school_summary['total_school_budget'] / school_summary['total_student']
school_summary['pass_math_ratio'] = round(school_summary['pass_math'] / school_summary['total_student'], 2)
school_summary['pass_read_ratio'] = round(school_summary['pass_read'] / school_summary['total_student'], 2)
school_summary['overall_pass_ratio'] = round((school_summary['pass_math_ratio'] + school_summary['pass_read_ratio']) / 2, 2)

top_5_overall_passing = school_summary.sort_values('overall_pass_ratio', ascending=False).head(5)

top_5_overall_passing

,school_name,type,total_student,total_school_budget,avg_math_score,avg_read_score,pass_math,pass_read,per_student_budget,pass_math_ratio,pass_read_ratio,overall_pass_ratio
1,Cabrera High School,Charter,1858,2009159448,83.061895,83.975780,1664,1744,1081356.0,0.90,0.94,0.92
4,Griffin High School,Charter,1468,1346890000,83.351499,83.816757,1317,1371,917500.0,0.90,0.93,0.92
6,Holden High School,Charter,427,105933149,83.803279,83.814988,387,396,248087.0,0.91,0.93,0.92
9,Pena High School,Charter,962,563595396,83.839917,84.044699,882,887,585858.0,0.92,0.92,0.92
11,Shelton High School,Charter,1761,1860672600,83.359455,83.725724,1583,1631,1056600.0,0.90,0.93,0.92


In [33]:
# Calculate total school budget
total_school_budget = school_data_complete['budget'].sum()

total_school_budget

82932329558

In [35]:
# Calculate per student budget

school_summary['per_student_budget'] = school_summary['total_school_budget'] / school_summary['total_student']

school_summary[['school_name', 'per_student_budget']]


,school_name,per_student_budget
0,Bailey High School,3124928.0
1,Cabrera High School,1081356.0
2,Figueroa High School,1884411.0
3,Ford High School,1763916.0
4,Griffin High School,917500.0
5,Hernandez High School,3022020.0
6,Holden High School,248087.0
7,Huang High School,1910635.0
8,Johnson High School,3094650.0
9,Pena High School,585858.0


In [37]:
# Cacluate the avg math and reading score
# Calculate average math and reading scores
avg_scores = school_summary[['avg_math_score', 'avg_read_score']]

avg_scores

,avg_math_score,avg_read_score
0,77.048432,81.033963
1,83.061895,83.975780
2,76.711767,81.158020
3,77.102592,80.746258
4,83.351499,83.816757
5,77.289752,80.934412
6,83.803279,83.814988
7,76.629414,81.182722
8,77.072464,80.966394
9,83.839917,84.044699


#### Find the passing rate for math and reading (above 70 points)

In [40]:
# Find the total counts of math result
total_math_count = school_data_complete['math_score'].count()


# Counts for math results in each school that pass 70 or higher
pass_math_counts = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name', 'type']).size().reset_index(name='pass_math_count')


# Calculate the math passing rate
math_passing_rate = (pass_math_counts['pass_math_count'] / total_math_count) * 100
pass_math_counts['math_passing_rate'] = math_passing_rate
pass_math_counts

,school_name,type,pass_math_count,math_passing_rate
0,Bailey High School,District,3318,8.470768
1,Cabrera High School,Charter,1749,4.465152
2,Figueroa High School,District,1946,4.968088
3,Ford High School,District,1871,4.776615
4,Griffin High School,Charter,1371,3.500128
5,Hernandez High School,District,3094,7.898902
6,Holden High School,Charter,395,1.008425
7,Huang High School,District,1916,4.891499
8,Johnson High School,District,3145,8.029104
9,Pena High School,Charter,910,2.323207


In [48]:
# Find the total counts of read result
total_read_count = school_data_complete['reading_score'].count()

# Find the counts for read result in each school that pass 70 or higher
pass_read_counts = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name', 'type']).size().reset_index(name='pass_read_count')

# Calculate the read passing rate
read_passing_rate = (pass_read_counts['pass_read_count'] / total_read_count) * 100
pass_read_counts['read_passing_rate'] = read_passing_rate
pass_read_counts


,school_name,type,pass_read_count,read_passing_rate
0,Bailey High School,District,4077,10.408476
1,Cabrera High School,Charter,1803,4.603013
2,Figueroa High School,District,2381,6.078632
3,Ford High School,District,2172,5.545060
4,Griffin High School,Charter,1426,3.640541
5,Hernandez High School,District,3748,9.568547
6,Holden High School,Charter,411,1.049272
7,Huang High School,District,2372,6.055655
8,Johnson High School,District,3867,9.872351
9,Pena High School,Charter,923,2.356395


In [50]:
# Calculate the overall passing rate (average of the math and reading passing rate)
overall_score = (avg_math + avg_read)/2
print(f'Overall average score: {overall_score}')



Overall average score: 80.43160582078121


### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [56]:
#  Sort and display the worst five schools in overall passing rate
school_summary.sort_values(['overall_pass_ratio', 
                            'pass_math_ratio', 
                            'pass_read_ratio']).head(5)

,school_name,type,total_student,total_school_budget,avg_math_score,avg_read_score,pass_math,pass_read,per_student_budget,pass_math_ratio,pass_read_ratio,overall_pass_ratio
7,Huang High School,District,2917,5573322295,76.629414,81.182722,1847,2299,1910635.0,0.63,0.79,0.71
2,Figueroa High School,District,2949,5557128039,76.711767,81.158020,1880,2313,1884411.0,0.64,0.78,0.71
8,Johnson High School,District,4761,14733628650,77.072464,80.966394,3040,3727,3094650.0,0.64,0.78,0.71
10,Rodriguez High School,District,3999,10186904637,76.842711,80.744686,2562,3109,2547363.0,0.64,0.78,0.71
5,Hernandez High School,District,4635,14007062700,77.289752,80.934412,3001,3624,3022020.0,0.65,0.78,0.72


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [88]:
# Calculate the average math score for 9th grade in each school

avg_math_9th = school_data_complete[school_data_complete['grade'] == '9th'].groupby('school_name')['math_score'].mean()


In [90]:
# Calculate the average math score for 10th grade in each school

avg_math_10th = school_data_complete[school_data_complete['grade'] == '10th'].groupby('school_name')['math_score'].mean()


In [92]:
# Calculate the average math score for 11th grade in each school

avg_math_11th = school_data_complete[school_data_complete['grade'] == '11th'].groupby('school_name')['math_score'].mean()


In [94]:
# Calculate the average math score for 12th grade in each school

avg_math_12th = school_data_complete[school_data_complete['grade'] == '12th'].groupby('school_name')['math_score'].mean()


In [96]:
# Create table that lists the average math score for each school of each grade level.
avg_math_by_grade = pd.DataFrame({
    '9th Grade': avg_math_9th,
    '10th Grade': avg_math_10th,
    '11th Grade': avg_math_11th,
    '12th Grade': avg_math_12th
})

avg_math_by_grade = avg_math_by_grade.round(2)
avg_math_by_grade



,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [100]:
# Calculate the average reading score for 9th grade in each school

avg_reading_9th = school_data_complete[school_data_complete['grade'] == '9th'].groupby('school_name')['reading_score'].mean()


In [102]:
# Calculate the average reading score for 10th grade in each school

avg_reading_10th = school_data_complete[school_data_complete['grade'] == '10th'].groupby('school_name')['reading_score'].mean()


In [104]:
# Calculate the average reading score for 11th grade in each school

avg_reading_11th = school_data_complete[school_data_complete['grade'] == '11th'].groupby('school_name')['reading_score'].mean()


In [106]:
# Calculate the average reading score for 12th grade in each school

avg_reading_12th = school_data_complete[school_data_complete['grade'] == '12th'].groupby('school_name')['reading_score'].mean()


In [108]:
# Create table that lists the average reading score for each school of each grade level.

avg_reading_by_grade = pd.DataFrame({
    '9th Grade': avg_reading_9th,
    '10th Grade': avg_reading_10th,
    '11th Grade': avg_reading_11th,
    '12th Grade': avg_reading_12th
})

avg_reading_by_grade = avg_reading_by_grade.round(2)
avg_reading_by_grade



,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [209]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 600, 625, 650, 1000]
group_names = ["<$585", "$585-600", "$600-625", "$625-650", "$650<"]

In [233]:
# Create a new column to show budget per student in each row

school_data_complete["spending per student"] = school_data_complete["budget"] / school_data_complete["size"]


In [235]:
# Create a new column to define the spending ranges per student

school_data_complete["spending"] = pd.cut(school_data_complete["spending per student"], spending_bins, labels=group_names)


In [239]:
# Calculate the average math score within each spending range

avg_score = school_data_complete.groupby("spending", observed=True).agg(
    avg_math=('math_score', 'mean'),
    avg_read=('reading_score', 'mean'),
    math_pass=('math_score', lambda x: (x >= 70).sum()),
    read_pass=('reading_score', lambda x: (x >= 70).sum()),
    total_math=('math_score', 'count'),
    total_read=('reading_score', 'count')
).reset_index()


In [243]:
# Calculate the percentage passing rate for math in each spending range

avg_score['math_percent_pass'] = avg_score['math_pass'] / avg_score['total_math']


In [245]:
# Calculate the percentage passing rate for reading in each spending range

avg_score['read_percent_pass'] = avg_score['read_pass'] / avg_score['total_read']


In [247]:
# Calculate the percentage overall passing rate in each spending range
avg_score['overall_score'] = (avg_score['math_percent_pass'] + avg_score['read_percent_pass']) / 2

# Rename columns for clarity
avg_score.rename(columns={
    'math_pass': 'math_pass',
    'read_pass': 'read_pass'
}, inplace=True)

avg_score

,spending,avg_math,avg_read,math_pass,read_pass,total_math,total_read,math_percent_pass,read_percent_pass,overall_score
0,<$585,83.363065,83.964039,5967,6157,6368,6368,0.937029,0.966866,0.951947
1,$585-600,83.359455,83.725724,1653,1688,1761,1761,0.938671,0.958546,0.948609
2,$600-625,83.544856,83.906996,2281,2349,2430,2430,0.938683,0.966667,0.952675
3,$625-650,77.469253,81.162258,14459,17296,21059,21059,0.686595,0.821312,0.753953
4,$650<,77.034693,81.030323,5010,6120,7552,7552,0.663400,0.810381,0.736891


### Scores by School Size

* Perform the same operations as above, based on school size.

In [184]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [198]:
# Create a new column for the bin groups
school_data_complete["School Size"] = pd.cut(school_data_complete["size"],
    size_bins, labels=group_names)

size_df = school_data_complete.groupby(["School Size"], observed=True)
avg_math_size = size_df["math_score"].mean()
avg_read_read = size_df["reading_score"].mean()
test_count_size = size_df["Student ID"].count()


Look for the total count of test scores that pass 70% or higher




In [203]:
# math_pass_size
math_pass_size = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
math_pass_size_df = math_pass_size.groupby(["School Size"], observed=True)
math_count_size = math_pass_size_df["math_score"].count()
math_pass_rate_size = math_count_size / test_count_size



In [205]:
# read_pass_size

read_pass_size = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
read_pass_size_df = read_pass_size.groupby(["School Size"], observed=True)
read_count_size = read_pass_size_df["reading_score"].count()
read_pass_rate_size = read_count_size / test_count_size


In [207]:
# Calculate the overall passing rate for different school size

overall_pass_rate_size = (math_pass_rate_size + read_pass_rate_size) / 2

school_size_df = pd.DataFrame({'math_pass':math_pass_rate_size ,
                               'read_pass':read_pass_rate_size, 
                               'overall_pass':overall_pass_rate_size}).reset_index()
school_size_df


,School Size,math_pass,read_pass,overall_pass
0,Small (<1000),0.939525,0.960403,0.949964
1,Medium (1000-2000),0.936165,0.967731,0.951948
2,Large (2000-5000),0.686524,0.821252,0.753888


### Scores by School Type

* Perform the same operations as above, based on school type.

In [249]:
# Create bins and groups, school type {'Charter', 'District'}
school_type = school_data_complete.groupby(["type"])
avg_math_type = school_type["math_score"].mean()
avg_read_type = school_type["reading_score"].mean()
test_count_type = school_type["Student ID"].count()


Find counts of the passing 70 or higher score for the both test


In [252]:
# math pass size

math_pass_rate_type = (school_data_complete["math_score"] >= 70).groupby(school_data_complete["type"]).mean()


In [254]:
# reading pass size
read_pass_rate_type = (school_data_complete["reading_score"] >= 70).groupby(school_data_complete["type"]).mean()


In [256]:
# Calculate the overall passing rate
overall_pass_rate_type = (math_pass_rate_type + read_pass_rate_type) / 2
pd.DataFrame({
    'math_pass': math_pass_rate_type,
    'read_pass': read_pass_rate_type,
    'overall_pass': overall_pass_rate_type
})


,math_pass,read_pass,overall_pass
type,,,
Charter,0.937018,0.966459,0.951739
District,0.665184,0.809052,0.737118
